## Idealised Hypothetical Disease Treatment

In [32]:
import joblib
import numpy as np
# import os
import plotly.graph_objects as go
import random
import torch
import wandb
from mclatte.model import train_mclatte, train_semi_skimmed_mclatte, train_skimmed_mclatte
from mclatte.simulation_data import generate_simulation_data, TreatmentRepr
from rnn.model import train_baseline_rnn
from synctwin.model import train_synctwin

In [33]:
random.seed(509)
np.random.seed(509)
torch.manual_seed(509)

Data Generation

In [34]:
M = 5
H = 5
R = 5
D = 10
K = 3
C = 4

In [35]:
def generate_data(N, p_0, mode, run_idx):
    data_path = f'data/test/idt_{N}_{p_0}_{mode}_{run_idx}.joblib'
    try:
        return joblib.load(data_path)
    except Exception as e:
        print(e)
        N_train = round(N * 0.8)
        N_test = round(N * 0.2)
        X, M_, Y_pre, Y_post, A, T = generate_simulation_data(N, M, H, R, D, K, C, mode, p_0)
        X_train, X_test = X[:N_train], X[N_train:]
        M_train, M_test = M_[:N_train], M_[N_train:]
        Y_pre_train, Y_pre_test = Y_pre[:N_train], Y_pre[N_train:]
        Y_post_train, Y_post_test = Y_post[:N_train], Y_post[N_train:]
        A_train, A_test = A[:N_train], A[N_train:]
        T_train, T_test = T[:N_train], T[N_train:]
        all_data = (
            N, N_train, N_test, 
            X_train, X_test, 
            M_train, M_test, 
            Y_pre_train, Y_pre_test, 
            Y_post_train, Y_post_test, 
            A_train, A_test, 
            T_train, T_test
        )
        joblib.dump(all_data, data_path)
        return all_data

In [36]:
# joblib.dump((N_train, M, H, R, D, K, C, X_train, M_train, Y_pre_train, Y_post_train, A_train, T_train), 'data/simulation/data_uniform_200.joblib')
# joblib.dump((N_test, M, H, R, D, K, C, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test), 'data/simulation/test_data_uniform_200.joblib')

### Visualizations

In [37]:
(
    N_visual, 
    N_train_visual, 
    N_test_visual, 
    X_train_visual, 
    X_test_visual, 
    M_train_visual, 
    M_test_visual, 
    Y_pre_train_visual, 
    Y_pre_test_visual, 
    Y_post_train_visual, 
    Y_post_test_visual, 
    A_train_visual, 
    A_test_visual, 
    T_train_visual, 
    T_test_visual,
) = generate_data(250, 0, TreatmentRepr.BINARY)
X_visual = np.concatenate((X_train_visual, X_test_visual), axis=0)
A_visual = np.concatenate((A_train_visual, A_test_visual), axis=0)
Y_pre_visual = np.concatenate((Y_pre_train_visual, Y_pre_test_visual), axis=0)
Y_post_visual = np.concatenate((Y_post_train_visual, Y_post_test_visual), axis=0)
sample_ids = np.random.randint(N_visual, size=10)

Covariates

In [38]:
fig = go.Figure()
for feature_idx in range(D):
    values = np.mean(X_visual[sample_ids, :, feature_idx], axis=0)
    fig.add_trace(go.Scatter(x=list(range(R * M)), y=values, name=f'feature {feature_idx}'))
fig.update_layout(
    title='Average Covariate Values',
    xaxis_title='t',
    yaxis_title='Feature Value'
)
fig.show()

Treatment Causes

In [39]:
fig = go.Figure(data=go.Heatmap(z=A_visual[sample_ids].T))
fig.update_layout(
    title='Treatment Causes',
    xaxis_title='Sample ID',
    yaxis_title='Cause'
)
fig.show()

Treatment Outcomes

In [40]:
Y_sampled = np.concatenate((Y_pre_visual, Y_post_visual), axis=1)[sample_ids, :]

In [41]:
fig = go.Figure()
for sample_idx in range(len(sample_ids)):
    values = Y_sampled[sample_idx, :]
    fig.add_trace(go.Scatter(x=list(range(M + H)), y=values, name=f'Sample {sample_idx}'))
fig.update_layout(
    title='Sampled Treatment Outcomes',
    xaxis_title='t',
    yaxis_title='Outcome Value'
)
fig.show()

## Modelling

In [42]:
wandb.init(project='mclatte-test', entity='jasonyz')

epoch,▁▂▃▄▁▂▃▄▅▅▆▇█▁▂▃▄▅▅▆▁▂▃▄▅▅▅▆▇██
ptl/loss,█▁
ptl/valid_loss,▂▃▃▃▇▆▇▆▆▆▆▆▆█▇▆▅▆▆▆▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▁▁▁▁▂▂▂▂▂▁▁▁▁▂▂▂▂▂▃▄▄▅▆▆▇▇█
epoch,8
ptl/loss,2.4435
ptl/valid_loss,1.34684
trainer/global_step,107


In [43]:
# N_train, M, H, R, D, K, C, X_train, M_train, Y_pre_train, Y_post_train, A_train, T_train = joblib.load(
#     os.path.join(os.getcwd(), f'data/simulation/data.joblib')
# )
# N_test, M, H, R, D, K, C, X_test, M_test, Y_pre_test, Y_post_test, A_test, T_test = joblib.load(
#     os.path.join(os.getcwd(), 'data/simulation/test_data.joblib')
# )

In [44]:
def na_catcher(func):
    def wrapper_na_catcher(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(e)
            return np.nan
    return wrapper_na_catcher

### McLatte

#### Skimmed

In [45]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results_idt/skimmed_mclatte_hp.csv')).sort_values(by='valid_loss').iloc[0])
skimmed_mclatte_config = {
    'encoder_class': 'lstm',
    'decoder_class': 'lstm',
    'hidden_dim': 64,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.0151,
    'gamma': 0.986855,
    'lambda_r': 1.928836,
    'lambda_p': 0.042385,
}

In [46]:
def test_skimmed_mclatte(
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    trained_skimmed_mclatte = train_skimmed_mclatte(
        skimmed_mclatte_config,
        X_train,
        M_train,
        Y_pre_train,
        Y_post_train,
        A_train, 
        T_train,
        R,
        M,
        H,
        input_dim=D, 
        treatment_dim=K, 
        test_run=run_idx,
    )

    trained_skimmed_mclatte.eval()
    _, y_tilde = trained_skimmed_mclatte(
        torch.from_numpy(X_test).float(),
        torch.from_numpy(A_test).float(),
        torch.from_numpy(T_test).float(),
        torch.from_numpy(M_test).float(),
    )
    
    return torch.nn.functional.l1_loss(
        y_tilde, 
        torch.from_numpy(Y_post_test).float()
    ).item()

#### Semi-Skimmed

In [47]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results_idt/semi_skimmed_mclatte_hp.csv')).sort_values(by='valid_loss').iloc[0])
semi_skimmed_mclatte_config = {
    'encoder_class': 'lstm',
    'decoder_class': 'lstm',
    'hidden_dim': 16,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.120062,
    'gamma': 0.731629,
    'lambda_r': 0.016767,
    'lambda_d': 1.83538,
    'lambda_p': 1.509965,
}

In [48]:
def test_semi_skimmed_mclatte(
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    trained_semi_skimmed_mclatte = train_semi_skimmed_mclatte(
        semi_skimmed_mclatte_config,
        X_train,
        M_train,
        Y_pre_train,
        Y_post_train,
        A_train, 
        T_train,
        R,
        M,
        H,
        input_dim=D, 
        treatment_dim=K, 
        test_run=run_idx,
    )

    trained_semi_skimmed_mclatte.eval()
    _, _, y_tilde = trained_semi_skimmed_mclatte(
        torch.from_numpy(X_test).float(),
        torch.from_numpy(A_test).float(),
        torch.from_numpy(T_test).float(),
        torch.from_numpy(M_test).float(),
    )
    
    return torch.nn.functional.l1_loss(
        y_tilde, 
        torch.from_numpy(Y_post_test).float()
    ).item()

#### Vanilla

In [49]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results_idt/mclatte_hp.csv')).sort_values(by='valid_loss').iloc[0])
mclatte_config = {
    'encoder_class': 'lstm',
    'decoder_class': 'lstm',
    'hidden_dim': 16,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.024468,
    'gamma': 0.740409,
    'lambda_r': 0.040299,
    'lambda_d': 0.034368,
    'lambda_p': 0.021351,
}

In [50]:
def test_mclatte(
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    trained_mclatte = train_mclatte(
        mclatte_config,
        X_train,
        M_train,
        Y_pre_train,
        Y_post_train,
        A_train, 
        T_train,
        R,
        M,
        H,
        input_dim=D, 
        treatment_dim=K, 
        test_run=run_idx,
    )

    trained_mclatte.eval()
    _, _, y_tilde = trained_mclatte(
        torch.from_numpy(X_test).float(),
        torch.from_numpy(A_test).float(),
        torch.from_numpy(T_test).float(),
        torch.from_numpy(M_test).float(),
    )
    
    return torch.nn.functional.l1_loss(
        y_tilde, 
        torch.from_numpy(Y_post_test).float()
    ).item()

### Baseline RNN

In [51]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results/baseline_rnn_hp.csv')).sort_values(by='valid_loss').iloc[0])
rnn_config = {
    'rnn_class': 'gru',
    'hidden_dim': 4,
    'seq_len': 4,
    'batch_size': 64,
    'epochs': 100,
    'lr': 0.048177,
    'gamma': 0.795612,
}

In [52]:
def rnn_predict(trained_rnn, Y_pre, Y_post):
    """
    Make predictions using results from previous time steps.
    """
    Y = Y_pre
    losses = 0.0
    for i in range(Y_post.shape[1]):
        Y_tilde = trained_rnn(
            torch.from_numpy(Y).float().unsqueeze(2)
        ).squeeze()

        Y = np.concatenate((
            Y[:, 1:], 
            Y_tilde.cpu().detach().numpy()[:, [-1]]
        ), axis=1)
        
        losses += torch.nn.functional.l1_loss(
            Y_tilde[:, -1], 
            torch.from_numpy(Y_post).float()[:, i]
        ).item()
    return losses / Y_post.shape[1]

In [53]:
def test_rnn(
    Y_pre_train, 
    Y_pre_test, 
    Y_post_train, 
    Y_post_test, 
    run_idx=0,
):
    trained_rnn = train_baseline_rnn(
        rnn_config,
        Y=np.concatenate((Y_pre_train, Y_post_train), axis=1),
        input_dim=1, 
        test_run=run_idx,
    )

    trained_rnn.eval()
    return rnn_predict(trained_rnn, Y_pre_test, Y_post_test)

### SyncTwin

#### Full

In [54]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results/synctwin_hp.csv')).sort_values(by='valid_loss').iloc[0])
synctwin_config = {
    'hidden_dim': 32,
    'reg_B': 0.909119,
    'lam_express': 0.106598,
    'lam_recon': 0.441844,
    'lam_prognostic': 0.207286,
    'tau': 0.311216,
    'batch_size': 32,
    'epochs': 100,
    'lr': 0.000196,
    'gamma': 0.888244,
}

In [55]:
def test_synctwin(
    N, 
    N_test, 
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    Y_mask_train = np.all(A_train == 0, axis=1)
    Y_mask_test = np.all(A_test == 0, axis=1)
    Y_control_train = Y_post_train[Y_mask_train]

    trained_synctwin = train_synctwin(
        synctwin_config,
        X=X_train,
        M_=M_train,
        T=T_train,
        Y_batch=Y_post_train,
        Y_control=Y_control_train,
        Y_mask=Y_mask_train, 
        N=N,
        D=D,
        n_treated=N - Y_control_train.shape[0],
        pre_trt_x_len=R * M,
        test_run=run_idx,
    ).cuda()

    trained_synctwin.eval()
    _, l1_loss = trained_synctwin(
        torch.from_numpy(X_test).float().cuda(),
        torch.from_numpy(T_test).float().cuda(),
        torch.from_numpy(M_test).float().cuda(),
        torch.arange(0, N_test).cuda(),
        torch.from_numpy(Y_post_test).float().cuda(),
        torch.from_numpy(Y_mask_test).float().cuda(),
    )
    return l1_loss.item()

#### Lambda S

In [56]:
lambda_s_synctwin_config = {
    'hidden_dim': 32,
    'reg_B': 0.909119,
    'lam_express': 0.106598,
    'lam_recon': 0,
    'lam_prognostic': 0.207286,
    'tau': 0.311216,
    'batch_size': 32,
    'epochs': 100,
    'lr': 0.000196,
    'gamma': 0.888244,
}

In [57]:
def test_lambda_s_synctwin(
    N, 
    N_test, 
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    Y_mask_train = np.all(A_train == 0, axis=1)
    Y_mask_test = np.all(A_test == 0, axis=1)
    Y_control_train = Y_post_train[Y_mask_train]

    trained_synctwin = train_synctwin(
        lambda_s_synctwin_config,
        X=X_train,
        M_=M_train,
        T=T_train,
        Y_batch=Y_post_train,
        Y_control=Y_control_train,
        Y_mask=Y_mask_train, 
        N=N,
        D=D,
        n_treated=N - Y_control_train.shape[0],
        pre_trt_x_len=R * M,
        test_run=run_idx,
    ).cuda()

    trained_synctwin.eval()
    _, l1_loss = trained_synctwin(
        torch.from_numpy(X_test).float().cuda(),
        torch.from_numpy(T_test).float().cuda(),
        torch.from_numpy(M_test).float().cuda(),
        torch.arange(0, N_test).cuda(),
        torch.from_numpy(Y_post_test).float().cuda(),
        torch.from_numpy(Y_mask_test).float().cuda(),
    )
    return l1_loss.item()

#### Lambda R

In [58]:
# print(pd.read_csv(os.path.join(os.getcwd(), 'results/synctwin_hp.csv')).sort_values(by='valid_loss').iloc[0])
lambda_r_synctwin_config = {
    'hidden_dim': 32,
    'reg_B': 0.909119,
    'lam_express': 0.106598,
    'lam_recon': 0.441844,
    'lam_prognostic': 0,
    'tau': 0.311216,
    'batch_size': 32,
    'epochs': 100,
    'lr': 0.000196,
    'gamma': 0.888244,
}

In [59]:
def test_lambda_r_synctwin(
    N, 
    N_test, 
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
    run_idx=0,
):
    Y_mask_train = np.all(A_train == 0, axis=1)
    Y_mask_test = np.all(A_test == 0, axis=1)
    Y_control_train = Y_post_train[Y_mask_train]

    trained_synctwin = train_synctwin(
        lambda_r_synctwin_config,
        X=X_train,
        M_=M_train,
        T=T_train,
        Y_batch=Y_post_train,
        Y_control=Y_control_train,
        Y_mask=Y_mask_train, 
        N=N,
        D=D,
        n_treated=N - Y_control_train.shape[0],
        pre_trt_x_len=R * M,
        test_run=run_idx,
    ).cuda()

    trained_synctwin.eval()
    _, l1_loss = trained_synctwin(
        torch.from_numpy(X_test).float().cuda(),
        torch.from_numpy(T_test).float().cuda(),
        torch.from_numpy(M_test).float().cuda(),
        torch.arange(0, N_test).cuda(),
        torch.from_numpy(Y_post_test).float().cuda(),
        torch.from_numpy(Y_mask_test).float().cuda(),
    )
    return l1_loss.item()

## Test Models

In [60]:
N_TEST = 5

In [61]:
TEST_CONFIGS = [
    [200, 0.1, TreatmentRepr.BINARY],
    [200, 0.5, TreatmentRepr.BINARY],
    [200, 0.1, TreatmentRepr.BOUNDED],
    [200, 0.5, TreatmentRepr.BOUNDED],
    [200, 0.1, TreatmentRepr.REAL_VALUED],
    [200, 0.5, TreatmentRepr.REAL_VALUED],
    # [1000, 0.1, TreatmentRepr.BINARY],
    # [1000, 0.5, TreatmentRepr.BINARY],
    # [1000, 0.1, TreatmentRepr.BOUNDED],
    # [1000, 0.5, TreatmentRepr.BOUNDED],
    # [1000, 0.1, TreatmentRepr.REAL_VALUED],
    # [1000, 0.5, TreatmentRepr.REAL_VALUED],
]

In [62]:
for config_idx in range(len(TEST_CONFIGS)):
    config = TEST_CONFIGS[config_idx]
    mclatte_losses = []
    semi_skimmed_mclatte_losses = []
    skimmed_mclatte_losses = []
    rnn_losses = []
    synctwin_losses = []
    for i in range(N_TEST * config_idx + 1, N_TEST * (1 + config_idx) + 1):
        (
            N, 
            N_train, 
            N_test, 
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_pre_test, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
        ) = generate_data(config[0], config[1], config[2], i)

        mclatte_losses.append(test_mclatte(
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))
        semi_skimmed_mclatte_losses.append(test_semi_skimmed_mclatte(
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))
        skimmed_mclatte_losses.append(test_skimmed_mclatte(
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_pre_train, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))

        rnn_losses.append(test_rnn(
            Y_pre_train, 
            Y_pre_test, 
            Y_post_train, 
            Y_post_test, 
            run_idx=i,
        ))

        synctwin_losses.append(test_synctwin(
            N_train, 
            N_test, 
            X_train, 
            X_test, 
            M_train, 
            M_test, 
            Y_post_train, 
            Y_post_test, 
            A_train, 
            A_test, 
            T_train, 
            T_test,
            run_idx=i,
        ))
        joblib.dump((
            config, 
            mclatte_losses, 
            semi_skimmed_mclatte_losses, 
            skimmed_mclatte_losses, 
            rnn_losses,
            synctwin_losses,
        ), f'results/test/config_{config_idx}_idt.joblib')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type        | Params
-----------------------------------------
0 | _encoder | LstmEncoder | 2.1 K 
1 | _decoder | LstmDecoder | 2.3 K 
-----------------------------------------
4.6 K     Trainable params
0         Non-trainable params
4.6 K     Total params
0.019     Total estimated model params size (MB)
Session not detected. You should not be calling `report` outside `tune.run` or while using the class API. 
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.2544.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.2544.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\Jason\Projects\McLatte\.venv.mclatte

#### Check finished runs results

In [ ]:
config, mclatte_losses, rnn_losses, synctwin_losses = joblib.load(f'results_idt/test/config_11.joblib')
config, mclatte_losses, synctwin_losses, rnn_losses

In [ ]:
for losses in mclatte_losses, synctwin_losses, rnn_losses:
    print(f'{np.mean(losses):.3f} ({np.std(losses):.3f})')

### Plot with trained models

In [ ]:
import os
import plotly.graph_objects as go
import torch
from mclatte.model import McLatte
from mclatte.simulation_data import generate_simulation_data, TreatmentRepr

In [ ]:
M = 5
H = 5
R = 5
D = 10
K = 3
C = 4
def generate_data(N, p_0, mode):
    N_train = round(N * 0.8)
    N_test = round(N * 0.2)
    X, M_, Y_pre, Y_post, A, T = generate_simulation_data(N, M, H, R, D, K, C, mode, p_0)
    X_train, X_test = X[:N_train], X[N_train:]
    M_train, M_test = M_[:N_train], M_[N_train:]
    Y_pre_train, Y_pre_test = Y_pre[:N_train], Y_pre[N_train:]
    Y_post_train, Y_post_test = Y_post[:N_train], Y_post[N_train:]
    A_train, A_test = A[:N_train], A[N_train:]
    T_train, T_test = T[:N_train], T[N_train:]
    return N, N_train, N_test, X_train, X_test, M_train, M_test, Y_pre_train, Y_pre_test, Y_post_train, Y_post_test, A_train, A_test, T_train, T_test

In [ ]:
(
    N, 
    N_train, 
    N_test, 
    X_train, 
    X_test, 
    M_train, 
    M_test, 
    Y_pre_train, 
    Y_pre_test, 
    Y_post_train, 
    Y_post_test, 
    A_train, 
    A_test, 
    T_train, 
    T_test,
) = generate_data(200, 0.1, TreatmentRepr.BINARY)
trained_mclatte = McLatte.load_from_checkpoint(os.path.join(os.getcwd(), f'results/mclatte_51.ckpt'))

In [ ]:
trained_mclatte.eval()
_, y_tilde = trained_mclatte(
    torch.from_numpy(X_test).float(),
    torch.from_numpy(A_test).float(),
    torch.from_numpy(T_test).float(),
    torch.from_numpy(M_test).float(),
)

In [ ]:
PLOT_SUB_ID = 5
y_pre_plot = Y_pre_test[PLOT_SUB_ID]
pre_t = list(range(y_pre_plot.shape[0]))

y_post_plot = [y_pre_plot[-1]] + list(Y_post_test[PLOT_SUB_ID])
y_pred_plot = [y_pre_plot[-1]] + list(y_tilde.detach().numpy()[PLOT_SUB_ID])
post_t = np.arange(len(y_post_plot)) + y_pre_plot.shape[0] - 1

fig = go.Figure()
line_pre_trt = go.Scatter(x=pre_t, y=y_pre_plot, name='pre-treatment')
line_gt_post_trt = go.Scatter(x=post_t, y=y_post_plot, name='post-treatment ground truth')
line_pred_post_trt = go.Scatter(x=post_t, y=y_pred_plot, name='post-treatment predictions')
fig.add_trace(line_pre_trt)
fig.add_trace(line_gt_post_trt)
fig.add_trace(line_pred_post_trt)

fig.update_layout(title='Treatment Outcome',
                  yaxis_title='Outcome', xaxis_title='Time')
fig.show()